In [0]:
%pip install JayDeBeApi

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType,DateType,ArrayType,StringType
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import jaydebeapi
import uuid
@udf (returnType=ArrayType(DateType()))
def getDates(strDate,freq,bno):
    dates=[strDate]
    bno-=1
    for i in range(0,bno):
        date=dates[i] + relativedelta(months=freq)
        dates.append(date)
    return dates
@udf (returnType=StringType())
def uuidGen():
    return str(uuid.uuid4())
@udf (returnType=DateType())
def nbdGen(n1,freq):
    return n1+ relativedelta(months=freq)

def updateNBD(contractid,companyid,nbd):
    cursor = connection.cursor()
    nbd=datetime.strptime(nbd,"%d/%m/%Y")
    query=f"Update [dbo].[contracts] set nextBillingDate='{nbd}' where id={contractid}  and companyid={companyid}"
    cursor.execute(query)
    connection.commit()
    cursor.close()

def createInvoices(billableItems1):
    invoices=spark.read.jdbc(url=jdbcURL,table='invoice',properties=connectionProperties)
    maxid=invoices.agg({'id':'max'}).collect()[0]['max(id)']
    if(maxid==None):
        maxid=0
    billableItems1=billableItems1.withColumn('id',f.col('id')+maxid).drop('Billings')
    display(billableItems1)
    billableItems1.printSchema()
    dirname = datetime.today().strftime('%Y-%m-%d')
    billableItems1.write.csv('abfss://raw@myappdl.dfs.core.windows.net/Contracts/Invoices_'+dirname+'/',mode='overwrite',header=True)
    billableItems1.write.jdbc(url=jdbcURL,mode='append',table='invoice',properties=connectionProperties)

def SelectionCriteria():
    lst=[]
    lstFive=[]
    lastNso=None
    results=spark.sql("select id,companyId companyId,nextBillingDate,billingFreq,DATEDIFF(MONTH, nextBillingDate, CURRENT_TIMESTAMP) Months,invoiceaddressid,serviceCharge from contracts where nextBillingDate<=CURRENT_TIMESTAMP and upper(billingstatus)='ACTIVE'\
        group by id,companyId,billingFreq,nextBillingDate,serviceCharge,invoiceaddressid order by companyId asc")
    
    results=results.withColumn('Billings',f.floor(f.col('Months')/f.col('billingFreq')).cast(IntegerType())+1)\
    .withColumn("invoiceDates",f.array(getDates(f.col('nextBillingDate'),f.col('billingFreq'),f.col('Billings')))[0])
    display(results)

    billableItems=results.withColumn('contractId',f.col('id')).withColumn('invoiceDate',f.explode(f.col('invoiceDates'))).\
        withColumn('id',f.row_number().over(Window.orderBy(f.col('contractId').asc()))).withColumn('uuid',uuidGen()).\
        withColumn('finalInvoiceNumber',f.lit(None).cast(IntegerType())).\
        select('id','finalInvoiceNumber','uuid','contractId','companyid','Billings','invoiceDate','invoiceaddressid',f.col('serviceCharge').alias('Revenue'))
        
    billableItems.printSchema()

    resultsByNSO = results.withColumn('nbd',nbdGen(f.col('invoiceDates')[f.size(f.col('invoiceDates'))-1],f.col('billingFreq'))).select('id','companyId','nbd').collect()
    createInvoices(billableItems)
    try:
        for i in resultsByNSO:
            contractid=i['id']
            companyid=i['companyId']
            nbd=i['nbd'].strftime("%d/%m/%Y")
            updateNBD(contractid,companyid,nbd)
    except :
        connection.rollback()
        raise ValueError('something went wrong')


myappdl_sas = dbutils.secrets.get("myApp_scope","SasToken")
spark.conf.set("fs.azure.account.auth.type.myappdl.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.myappdl.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.myappdl.dfs.core.windows.net", myappdl_sas)
spark.conf.set("Spark.jars","dbfs:/FileStore/mssql_jdbc_12_2_0_jre11.jar")
jdbcHostname=dbutils.secrets.get("myApp_scope","jdbcHostname")
jdbcDatabase=dbutils.secrets.get("myApp_scope","jdbcDatabase")
jdbcPassword=dbutils.secrets.get("myApp_scope","jdbcPassword")
jdbcUsername=dbutils.secrets.get("myApp_scope","jdbcUsername")
jdbcURL='jdbc:sqlserver://'+jdbcHostname+':1433;database={'+jdbcDatabase+'}'
connectionProperties ={
    "user":jdbcUsername,
    "password":jdbcPassword,
    "driver":"com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
connection = jaydebeapi.connect(
    'com.microsoft.sqlserver.jdbc.SQLServerDriver',
    jdbcURL,
    {'user': jdbcUsername, 'password': jdbcPassword},  # Connection properties
    jars='dbfs:/FileStore/mssql_jdbc_12_2_0_jre11.jar'    # Path to the JDBC driver JAR file
)
connection.jconn.setAutoCommit(True)
contracts=spark.read.jdbc(url=jdbcURL,table='Contracts',properties=connectionProperties)
contracts.createOrReplaceTempView("contracts")
SelectionCriteria()
connection.close()